# Train Model with DPO

Code authored by: Shaw Talebi

### imports

In [1]:
import unsloth
from datasets import load_from_disk
from trl import DPOConfig, DPOTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
import torch
from peft import LoraConfig, get_peft_model, PeftModel, prepare_model_for_kbit_training

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Skipping import of cpp extensions due to incompatible torch version 2.8.0+cu128 for torchao version 0.14.0         Please see GitHub issue #2919 for more info


🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
dataset = load_from_disk("/home/ubuntu/projek_chatbot_galang/rlhf/data_prep/dataset/preferences_chatbot_hf_v2")
dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 450
    })
    valid: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 50
    })
})

### load model

In [3]:
model_name = "models/merged-taxbot-SeaLLMs-v3-1.5B_v9-DPO-v1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_storage=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    dtype=torch.float16,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token 

INFO:accelerate.utils.modeling: We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


In [4]:
model = prepare_model_for_kbit_training(model)

print("✅ Model loaded and prepared for training")

✅ Model loaded and prepared for training


In [5]:
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    target_modules=[
        "q_proj", 
        "v_proj", 
        "k_proj", 
        "o_proj",
        # "gate_proj",  
        # "up_proj",
        # "down_proj"
    ],
    task_type="CAUSAL_LM"
)

In [6]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 2,179,072 || all params: 1,545,893,376 || trainable%: 0.1410


### generate title with base model

In [ ]:
def format_chat_prompt(prompt_messages):
    """
    Convert prompt (list of role-content dicts) to string with chat tags.
    """
    formatted = ""
    for msg in prompt_messages:
        formatted += f"<|im_start|>{msg['role']}\n{msg['content']}<|im_end|>\n"
    formatted += "<|im_start|>assistant\n"  
    return formatted

In [ ]:
print("Example prompt:\n", format_chat_prompt(dataset["valid"][0]["prompt"]))

Example prompt:
 <|im_start|>system

    Jawab pertanyaan berdasarkan konteks berikut:
    
Konteks 1:
UU PPh Konsolidasi setelah UU 6 Tahun 2023 Pasal 5 Ayat 3
(Sumber: (Sumber : UU 10 Tahun 1994, Tanggal Berlaku : 1 Jan 1995), Skor: 7.76)

Isi dan/atau Penjelasan:
Isi: Dalam menentukan besarnya laba suatu bentuk usaha tetap: a. biaya administrasi kantor pusat yang diperbolehkan untuk dibebankan adalah biaya yang berkaitan dengan usaha atau kegiatan bentuk usaha tetap, yang besarnya ditetapkan oleh Direktur Jenderal Pajak; b. pembayaran kepada kantor pusat yang tidak diperbolehkan dibebankan sebagai biaya adalah: 1. royalti atau imbalan lainnya sehubungan dengan penggunaan harta, paten, atau hak-hak lainnya; 2. imbalan sehubungan
            



Konteks 2:
UU PPh Konsolidasi setelah UU 6 Tahun 2023 Pasal 5 Ayat 3
(Sumber: (Sumber : UU 10 Tahun 1994, Tanggal Berlaku : 1 Jan 1995), Skor: 9.04)

Isi dan/atau Penjelasan:
tetap merupakan satu kesatuan dengan kantor pusatnya, sehingga pemba

In [ ]:
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")
prompt_example = format_chat_prompt(dataset["valid"][0]["prompt"])
outputs = generator(prompt_example, max_length=256, temperature=0.7, do_sample=True)
print("Generated output:\n", outputs[0]["generated_text"])

Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated output:
 <|im_start|>system

    Jawab pertanyaan berdasarkan konteks berikut:
    
Konteks 1:
UU PPh Konsolidasi setelah UU 6 Tahun 2023 Pasal 5 Ayat 3
(Sumber: (Sumber : UU 10 Tahun 1994, Tanggal Berlaku : 1 Jan 1995), Skor: 7.76)

Isi dan/atau Penjelasan:
Isi: Dalam menentukan besarnya laba suatu bentuk usaha tetap: a. biaya administrasi kantor pusat yang diperbolehkan untuk dibebankan adalah biaya yang berkaitan dengan usaha atau kegiatan bentuk usaha tetap, yang besarnya ditetapkan oleh Direktur Jenderal Pajak; b. pembayaran kepada kantor pusat yang tidak diperbolehkan dibebankan sebagai biaya adalah: 1. royalti atau imbalan lainnya sehubungan dengan penggunaan harta, paten, atau hak-hak lainnya; 2. imbalan sehubungan
            



Konteks 2:
UU PPh Konsolidasi setelah UU 6 Tahun 2023 Pasal 5 Ayat 3
(Sumber: (Sumber : UU 10 Tahun 1994, Tanggal Berlaku : 1 Jan 1995), Skor: 9.04)

Isi dan/atau Penjelasan:
tetap merupakan satu kesatuan dengan kantor pusatnya, sehingga pem

### train model

In [7]:
ft_model_name = "taxbot-SeaLLMs-v3-1.5B_v9-DPO-v2"

training_args = DPOConfig(
    output_dir= f"models/{ft_model_name}", 
    logging_steps=25,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_strategy="epoch",
    eval_strategy="epoch",
    eval_steps=1,
    bf16=False,
    fp16=True,   
    report_to='none'
)

In [ ]:
trainer = DPOTrainer(
    model=model, 
    args=training_args, 
    processing_class=tokenizer, 
    train_dataset=dataset['train'],
    eval_dataset=dataset['valid'],
    peft_config=peft_config
)
trainer.train()

/home/ubuntu/projek_chatbot_galang/venv/lib/python3.10/site-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
/home/ubuntu/projek_chatbot_galang/venv/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 151645, 'bos_token_id': None, 'pad_token_id': 151645}.
The model is already on multiple devices. Skipping the move to device specified in `args`.


Epoch,Training Loss,Validation Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss


In [ ]:
trainer.evaluate()

{'eval_loss': 0.4330650269985199,
 'eval_runtime': 41.0536,
 'eval_samples_per_second': 1.218,
 'eval_steps_per_second': 0.609,
 'eval_rewards/chosen': -0.0639238953590393,
 'eval_rewards/rejected': -1.3103232383728027,
 'eval_rewards/accuracies': 0.8399999737739563,
 'eval_rewards/margins': 1.2463992834091187,
 'eval_logps/chosen': -186.7700653076172,
 'eval_logps/rejected': -206.8123016357422,
 'eval_logits/chosen': -1.7953916788101196,
 'eval_logits/rejected': -1.7955176830291748,
 'epoch': 3.0}

In [ ]:
trainer.save_model()

In [ ]:
base = AutoModelForCausalLM.from_pretrained(model_name, dtype="float16")
lora = PeftModel.from_pretrained(base, ft_model_name)

merged = lora.merge_and_unload()

merged.save_pretrained(f"./merged-{ft_model_name}")
tokenizer.save_pretrained(f"./merged-{ft_model_name}")

('./merged-taxbot-SeaLLMs-v3-1.5B_v9-DPO-v1/tokenizer_config.json',
 './merged-taxbot-SeaLLMs-v3-1.5B_v9-DPO-v1/special_tokens_map.json',
 './merged-taxbot-SeaLLMs-v3-1.5B_v9-DPO-v1/chat_template.jinja',
 './merged-taxbot-SeaLLMs-v3-1.5B_v9-DPO-v1/vocab.json',
 './merged-taxbot-SeaLLMs-v3-1.5B_v9-DPO-v1/merges.txt',
 './merged-taxbot-SeaLLMs-v3-1.5B_v9-DPO-v1/added_tokens.json',
 './merged-taxbot-SeaLLMs-v3-1.5B_v9-DPO-v1/tokenizer.json')